<a href="https://colab.research.google.com/github/arthursl12/POC1/blob/main/POC1_Turbofan_FD003_Model_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import glob

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor

from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PolynomialFeatures

In [3]:
sns.set_palette('colorblind')

# Data Preparation

In [4]:
# Dataset Download 
os.system('git clone https://github.com/arthursl12/dataset_2')
os.system('mv /content/dataset_2/CMaps /content/CMaps')
os.system('mv /content/dataset_2/data_processing /content/data_processing')
os.system('rm -rf dataset_2')

0

In [5]:
from data_processing.processing import DatasetProcessing
from data_processing.training import HyperparameterSearch, reclipper_scorer
from data_processing.eval import Evaluation

In [6]:
proc = DatasetProcessing()

## Data Integration

The data are provided as a zip-compressed text file with 26 columns of numbers, separated by spaces. Each row is a snapshot of data taken during a single operational cycle, each column is a different variable. The columns correspond to:  

1) unit number   
2) time, in cycles  
3) operational setting 1  
4) operational setting 2  
5) operational setting 3    
6) sensor measurement 1    
7) sensor measurement 2  
...  
26) sensor measurement 20


There are 6 conditions (or combinations) which the 3 operational settings can take.  
Condition 1: Altitude = 0, Mach Number = 0, TRA = 100  
Condition 2: Altitude = 10, Mach Number = 0.25, TRA = 100  
Condition 3: Altitude = 20, Mach Number = 0.7 TRA = 100  
Condition 4: Altitude = 25, Mach Number = 0.62, TRA = 60  
Condition 5: Altitude = 35 Mach Number = 0.84, TRA = 100  
Condition 6: Altitude = 42, Mach Number = 0.84, TRA = 100  
  
There is slight variation in all these conditions so you may get numbers like 24.453 instead of 25 exactly.

FD001: Condition 1 only  
FD002: Mix of all the conditions  
FD003: Condition 1 only  
FD004: Mix of all conditions  


In [7]:
index_cols, settings_cols, sensors_cols, cols = proc.column_names()
train, test, y_test = proc.read_dataset(3)
train

,unit_number,time,op_1,op_2,op_3,s_0,s_1,s_2,s_3,s_4,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,1,1,-0.0005,0.0004,100.0,518.67,642.36,1583.23,1396.84,14.62,...,522.31,2388.01,8145.32,8.4246,0.03,391,2388,100.0,39.11,23.3537
1,1,2,0.0008,-0.0003,100.0,518.67,642.50,1584.69,1396.89,14.62,...,522.42,2388.03,8152.85,8.4403,0.03,392,2388,100.0,38.99,23.4491
2,1,3,-0.0014,-0.0002,100.0,518.67,642.18,1582.35,1405.61,14.62,...,522.03,2388.00,8150.17,8.3901,0.03,391,2388,100.0,38.85,23.3669
3,1,4,-0.0020,0.0001,100.0,518.67,642.92,1585.61,1392.27,14.62,...,522.49,2388.08,8146.56,8.3878,0.03,392,2388,100.0,38.96,23.2951
4,1,5,0.0016,0.0000,100.0,518.67,641.68,1588.63,1397.65,14.62,...,522.58,2388.03,8147.80,8.3869,0.03,392,2388,100.0,39.14,23.4583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24715,100,148,-0.0016,-0.0003,100.0,518.67,643.78,1596.01,1424.11,14.62,...,519.66,2388.30,8138.08,8.5036,0.03,394,2388,100.0,38.44,22.9631
24716,100,149,0.0034,-0.0003,100.0,518.67,643.29,1596.38,1429.14,14.62,...,519.91,2388.28,8144.36,8.5174,0.03,395,2388,100.0,38.50,22.9746
24717,100,150,-0.0016,0.0004,100.0,518.67,643.84,1604.53,1431.41,14.62,...,519.44,2388.24,8135.95,8.5223,0.03,396,2388,100.0,38.39,23.0682
24718,100,151,-0.0023,0.0004,100.0,518.67,643.94,1597.56,1426.57,14.62,...,520.01,2388.26,8141.24,8.5148,0.03,395,2388,100.0,38.31,23.0753


## Preprocessing

### Test Set Transformation 
Test set has samples for all cycles, but has annotations only for last one

In [8]:
test.shape, y_test.shape

((16596, 26), (100, 1))

In [9]:
test_last = proc.transform_test(test)
test_last.head()

,s_0,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,s_9,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,518.67,642.59,1592.40,1409.87,14.62,21.58,560.53,2388.22,9085.50,1.31,...,528.05,2388.23,8158.77,8.2966,0.03,393,2388,100.0,39.43,23.5679
1,518.67,642.56,1587.42,1409.69,14.62,21.61,553.33,2388.18,9050.97,1.30,...,520.90,2388.17,8128.04,8.4514,0.03,392,2388,100.0,38.83,23.2821
2,518.67,642.75,1591.93,1417.66,14.62,21.60,563.61,2388.31,9091.69,1.31,...,531.36,2388.33,8173.56,8.3057,0.03,395,2388,100.0,39.27,23.6440
3,518.67,642.28,1584.68,1406.56,14.62,21.61,552.75,2388.07,9048.23,1.30,...,521.27,2388.09,8133.78,8.4337,0.03,392,2388,100.0,38.70,23.3804
4,518.67,642.15,1580.59,1397.26,14.62,21.58,553.82,2387.96,9050.89,1.30,...,521.74,2387.96,8132.51,8.3900,0.03,390,2388,100.0,38.89,23.4463


In [10]:
X_test = test_last

### Remaining Useful Life (RUL)

In [11]:
train = proc.add_remaining_useful_life_linear(train)
train[index_cols+['RUL']].head()

,unit_number,time,RUL
0,1,1,258
1,1,2,257
2,1,3,256
3,1,4,255
4,1,5,254


## Attributes and target separation

In [12]:
X_train, y_train = proc.X_y_train_divide(train)

In [13]:
y_train.head()

,RUL
0,258
1,257
2,256
3,255
4,254


In [14]:
X_train.head()

,s_0,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,s_9,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,518.67,642.36,1583.23,1396.84,14.62,21.61,553.97,2387.96,9062.17,1.3,...,522.31,2388.01,8145.32,8.4246,0.03,391,2388,100.0,39.11,23.3537
1,518.67,642.50,1584.69,1396.89,14.62,21.61,554.55,2388.00,9061.78,1.3,...,522.42,2388.03,8152.85,8.4403,0.03,392,2388,100.0,38.99,23.4491
2,518.67,642.18,1582.35,1405.61,14.62,21.61,554.43,2388.03,9070.23,1.3,...,522.03,2388.00,8150.17,8.3901,0.03,391,2388,100.0,38.85,23.3669
3,518.67,642.92,1585.61,1392.27,14.62,21.61,555.21,2388.00,9064.57,1.3,...,522.49,2388.08,8146.56,8.3878,0.03,392,2388,100.0,38.96,23.2951
4,518.67,641.68,1588.63,1397.65,14.62,21.61,554.74,2388.04,9076.14,1.3,...,522.58,2388.03,8147.80,8.3869,0.03,392,2388,100.0,39.14,23.4583


## Training and Evaluation functions

In [15]:
eval = Evaluation()

In [16]:
search = HyperparameterSearch()

# DecisionTreeRegressor

## Linear RUL

In [17]:
model = Pipeline([
    ('tree_reg'  ,   DecisionTreeRegressor(random_state=42))
])

In [18]:
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "tree_reg__criterion": ["squared_error","friedman_mse","absolute_error","poisson"],
        "tree_reg__splitter": ["best", "random"],
        "tree_reg__max_depth": [None,100,200,300],
        "tree_reg__min_samples_split": [2,5,10,15,20],
        "tree_reg__min_samples_leaf": [1,2,5,10,15,20,50,100],
        "tree_reg__max_features": ["sqrt", "log2"],
        "tree_reg__min_impurity_decrease": list(np.arange(0,150)/10),
        "tree_reg__ccp_alpha": list(np.round(np.linspace(0, 2, 81), decimals=3)),
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                            scorer='r2', ignore_warnings=True)
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 24720
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 49
n_resources: 500
Fitting 5 folds for each of 49 candidates, totalling 245 fits
----------
iter: 1
n_candidates: 17
n_resources: 1500
Fitting 5 folds for each of 17 candidates, totalling 85 fits
----------
iter: 2
n_candidates: 6
n_resources: 4500
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'tree_reg__splitter': 'best', 'tree_reg__min_samples_split': 20, 'tree_reg__min_samples_leaf': 50, 'tree_reg__min_impurity_decrease': 6.7, 'tree_reg__max_features': 'sqrt', 'tree_reg__max_depth': 300, 'tree_reg__criterion': 'friedman_mse', 'tree_reg__ccp_alpha': 1.55}
Pipeline(steps=[('tree_reg',
                 DecisionTreeRegressor(ccp_alpha=1.55, criterion='friedman_mse',
 

Best Model:
```
DecisionTreeRegressor(ccp_alpha=1.55, criterion='friedman_mse',
                        max_depth=300, max_features='sqrt',
                        min_impurity_decrease=6.7,
                        min_samples_leaf=50,
                        min_samples_split=20,
                        random_state=42))
```

In [19]:
model = Pipeline([
    ('tree_reg'  ,   DecisionTreeRegressor(ccp_alpha=1.55, criterion='friedman_mse',
                        max_depth=300, max_features='sqrt',
                        min_impurity_decrease=6.7,
                        min_samples_leaf=50,
                        min_samples_split=20,
                        random_state=42))
])
model

Pipeline(steps=[('tree_reg',
                 DecisionTreeRegressor(ccp_alpha=1.55, criterion='friedman_mse',
                                       max_depth=300, max_features='sqrt',
                                       min_impurity_decrease=6.7,
                                       min_samples_leaf=50,
                                       min_samples_split=20,
                                       random_state=42))])

In [20]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=0.656,RMSE=-57.973
(CV) R2=0.498,RMSE=-68.130


In [21]:
eval.show_result(y_test, model.predict(X_test))

R2=-0.630,RMSE=-52.853


Worse than simple linear regression

## Non-linear RUL

In [22]:
model = Pipeline([
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = DecisionTreeRegressor(random_state=42),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':50})))
    ])

In [23]:
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1),
        "trf_reg__regressor__criterion": ["squared_error","friedman_mse","absolute_error","poisson"],
        "trf_reg__regressor__splitter": ["best", "random"],
        "trf_reg__regressor__max_depth": [None,100,200,300],
        "trf_reg__regressor__min_samples_split": [2,5,10,15,20],
        "trf_reg__regressor__min_samples_leaf": [1,2,5,10,15,20,50,100],
        "trf_reg__regressor__max_features": ["sqrt", "log2"],
        "trf_reg__regressor__min_impurity_decrease": list(np.arange(0,150)/10),
        "trf_reg__regressor__ccp_alpha": list(np.round(np.linspace(0, 2, 81), decimals=3)) 
    
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions)
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 24720
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 49
n_resources: 500
Fitting 5 folds for each of 49 candidates, totalling 245 fits
----------
iter: 1
n_candidates: 17
n_resources: 1500
Fitting 5 folds for each of 17 candidates, totalling 85 fits
----------
iter: 2
n_candidates: 6
n_resources: 4500
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 98}, 'trf_reg__regressor__splitter': 'best', 'trf_reg__regressor__min_samples_split': 2, 'trf_reg__regressor__min_samples_leaf': 10, 'trf_reg__regressor__min_impurity_decrease': 5.5, 'trf_reg__regressor__max_features': 'sqrt', 'trf_reg__regressor__max_depth': 200, 'trf_reg__regressor__criterion': 'friedman_mse', 'trf_reg__r

Best Model:

```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 98}}

DecisionTreeRegressor(ccp_alpha=0.375,
                    criterion='friedman_mse',
                    max_depth=200,
                    max_features='sqrt',
                    min_impurity_decrease=5.5,
                    min_samples_leaf=10,
                    random_state=42),
```



In [24]:
model = Pipeline([
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = DecisionTreeRegressor(ccp_alpha=0.375,
                            criterion='friedman_mse',
                            max_depth=200,
                            max_features='sqrt',
                            min_impurity_decrease=5.5,
                            min_samples_leaf=10,
                            random_state=42),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':98})))
    ])
model

Pipeline(steps=[('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=DecisionTreeRegressor(ccp_alpha=0.375,
                                                                            criterion='friedman_mse',
                                                                            max_depth=200,
                                                                            max_features='sqrt',
                                                                            min_impurity_decrease=5.5,
                                                                            min_samples_leaf=10,
                                                                            random_state=42),
                                            transformer=FunctionTransformer(func=<function clip at 0x7efea91cec20>,
                                                                            kw_args={'a_max': 98,
                

In [25]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=0.840,RMSE=-12.016
(CV) R2=0.807,RMSE=-13.082


In [26]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))

R2=0.756,RMSE=-15.577


The best result we had so far in training and in testing. Very same model as FD001.

## PolyFeatures + Linear RUL

In [27]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('tree_reg'  ,   DecisionTreeRegressor(random_state=42))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures()),
                ('tree_reg', DecisionTreeRegressor(random_state=42))])

In [28]:
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False],
        "tree_reg__criterion": ["squared_error","friedman_mse","absolute_error","poisson"],
        "tree_reg__splitter": ["best", "random"],
        "tree_reg__max_depth": [None,100,200,300],
        "tree_reg__min_samples_split": [2,5,10,15,20],
        "tree_reg__min_samples_leaf": [1,2,5,10,15,20,50,100],
        "tree_reg__max_features": [None, "sqrt", "log2"],
        "tree_reg__min_impurity_decrease": list(np.arange(0,150)/10),
        "tree_reg__ccp_alpha": list(np.round(np.linspace(0, 2, 81), decimals=3)) 
    }
    model = search.run_HR_GS(model, X_train, y_train, 
                            param_distributions, scorer='r2')
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 24720
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 49
n_resources: 500
Fitting 5 folds for each of 49 candidates, totalling 245 fits
----------
iter: 1
n_candidates: 17
n_resources: 1500
Fitting 5 folds for each of 17 candidates, totalling 85 fits
----------
iter: 2
n_candidates: 6
n_resources: 4500
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'tree_reg__splitter': 'best', 'tree_reg__min_samples_split': 20, 'tree_reg__min_samples_leaf': 50, 'tree_reg__min_impurity_decrease': 7.1, 'tree_reg__max_features': 'log2', 'tree_reg__max_depth': 200, 'tree_reg__criterion': 'squared_error', 'tree_reg__ccp_alpha': 1.425, 'poly_ft__interaction_only': True, 'poly_ft__include_bias': False, 'poly_ft__degree': 2}
Pipeline(steps=[('po

Best Model:
```
PolynomialFeatures(include_bias=False, interaction_only=True)),
DecisionTreeRegressor(ccp_alpha=1.425, max_depth=200,
                        max_features='log2',
                        min_impurity_decrease=7.1,
                        min_samples_leaf=50,
                        min_samples_split=20,
                        random_state=42)
```


In [29]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(include_bias=False, interaction_only=True)),
    ('tree_reg'  ,   DecisionTreeRegressor(ccp_alpha=1.425, max_depth=200,
                        max_features='log2',
                        min_impurity_decrease=7.1,
                        min_samples_leaf=50,
                        min_samples_split=20,
                        random_state=42))
])
model

Pipeline(steps=[('poly_ft',
                 PolynomialFeatures(include_bias=False, interaction_only=True)),
                ('tree_reg',
                 DecisionTreeRegressor(ccp_alpha=1.425, max_depth=200,
                                       max_features='log2',
                                       min_impurity_decrease=7.1,
                                       min_samples_leaf=50,
                                       min_samples_split=20,
                                       random_state=42))])

In [30]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=0.622,RMSE=-60.761
(CV) R2=0.526,RMSE=-66.395


In [31]:
eval.show_result(y_test, model.predict(X_test))

R2=-0.504,RMSE=-50.765


No clear benefit from polynomial features in linear RUL, as with the other models. Actually, it worsened the results.

## PolyFeatures + Non-Linear RUL

In [32]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = DecisionTreeRegressor(random_state=42),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':96})))
])

In [33]:
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(70,150,1),
        "trf_reg__regressor__criterion": ["squared_error","friedman_mse","absolute_error","poisson"],
        "trf_reg__regressor__splitter": ["best", "random"],
        "trf_reg__regressor__max_depth": [None,100,200,300],
        "trf_reg__regressor__min_samples_split": [2,5,10,15,20],
        "trf_reg__regressor__min_samples_leaf": [1,2,5,10,15,20,50,100],
        "trf_reg__regressor__max_features": [None, "sqrt", "log2"],
        "trf_reg__regressor__min_impurity_decrease": list(np.arange(0,150)/10),
        "trf_reg__regressor__ccp_alpha": list(np.round(np.linspace(0, 2, 81), decimals=3)) 
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions)
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 24720
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 49
n_resources: 500
Fitting 5 folds for each of 49 candidates, totalling 245 fits
----------
iter: 1
n_candidates: 17
n_resources: 1500
Fitting 5 folds for each of 17 candidates, totalling 85 fits
----------
iter: 2
n_candidates: 6
n_resources: 4500
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 95}, 'trf_reg__regressor__splitter': 'random', 'trf_reg__regressor__min_samples_split': 15, 'trf_reg__regressor__min_samples_leaf': 15, 'trf_reg__regressor__min_impurity_decrease': 0.3, 'trf_reg__regressor__max_features': None, 'trf_reg__regressor__max_depth': 100, 'trf_reg__regressor__criterion': 'friedman_mse', 'trf_reg__

Best Model:
```
'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 95}
PolynomialFeatures(include_bias=False, interaction_only=True))
DecisionTreeRegressor(ccp_alpha=0.6,
                        criterion='friedman_mse',
                        max_depth=100,
                        min_impurity_decrease=0.3,
                        min_samples_leaf=15,
                        min_samples_split=15,
                        random_state=42,
                        splitter='random'),
```


In [34]:
model = Pipeline([
    ('poly_ft', PolynomialFeatures(include_bias=False, interaction_only=True)),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = DecisionTreeRegressor(ccp_alpha=0.6,
                                            criterion='friedman_mse',
                                            max_depth=100,
                                            min_impurity_decrease=0.3,
                                            min_samples_leaf=15,
                                            min_samples_split=15,
                                            random_state=42,
                                            splitter='random'),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':95})))
])
model

Pipeline(steps=[('poly_ft',
                 PolynomialFeatures(include_bias=False, interaction_only=True)),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=DecisionTreeRegressor(ccp_alpha=0.6,
                                                                            criterion='friedman_mse',
                                                                            max_depth=100,
                                                                            min_impurity_decrease=0.3,
                                                                            min_samples_leaf=15,
                                                                            min_samples_split=15,
                                                                            random_state=42,
                                                                            splitter='random'),
                           

In [35]:
# ~3min
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=0.842,RMSE=-11.475
(CV) R2=0.811,RMSE=-12.407


In [36]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))

R2=0.776,RMSE=-14.454


This time, it improved a little bit the performance.

# RandomForestRegressor

## Linear RUL

In [37]:
model = Pipeline([
    ('tree_reg'  ,   RandomForestRegressor(random_state=42, n_jobs=-1))
])

In [38]:
# We need specific shape 1D arrays for this model
y_train = np.array(y_train).ravel()

In [39]:
# ~3min
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "tree_reg__n_estimators": [50,100,200],
        "tree_reg__criterion": ['squared_error','absolute_error','poisson'],
        "tree_reg__max_depth": [None,50, 100, 150],
        "tree_reg__min_samples_split": [2,5,10,15,20],
        "tree_reg__min_samples_leaf": [1,2,5,10,15,20,50,100],
        "tree_reg__max_features": ["sqrt", "log2"],
        "tree_reg__min_impurity_decrease": list(np.arange(0,150)/10),
        "tree_reg__ccp_alpha": list(np.round(np.linspace(0, 2, 81), decimals=3)),
        "tree_reg__oob_score": [True, False]
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                            scorer='r2', ignore_warnings=True)
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 24720
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 49
n_resources: 500
Fitting 5 folds for each of 49 candidates, totalling 245 fits
----------
iter: 1
n_candidates: 17
n_resources: 1500
Fitting 5 folds for each of 17 candidates, totalling 85 fits
----------
iter: 2
n_candidates: 6
n_resources: 4500
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'tree_reg__oob_score': True, 'tree_reg__n_estimators': 200, 'tree_reg__min_samples_split': 2, 'tree_reg__min_samples_leaf': 15, 'tree_reg__min_impurity_decrease': 0.6, 'tree_reg__max_features': 'log2', 'tree_reg__max_depth': 50, 'tree_reg__criterion': 'squared_error', 'tree_reg__ccp_alpha': 0.875}
Pipeline(steps=[('tree_reg',
                 RandomForestRegressor(ccp_alpha=0.8

Best Model:
```
RandomForestRegressor(ccp_alpha=0.875, max_depth=50,
                    max_features='log2',
                    min_impurity_decrease=0.6,
                    min_samples_leaf=15, n_estimators=200,
                    n_jobs=-1, oob_score=True,
                    random_state=42))
```

In [40]:
model = Pipeline([
    ('tree_reg'  ,   RandomForestRegressor(ccp_alpha=0.875, max_depth=50,
                            max_features='log2',
                            min_impurity_decrease=0.6,
                            min_samples_leaf=15, n_estimators=200,
                            n_jobs=-1, oob_score=True,
                            random_state=42))
])
model

Pipeline(steps=[('tree_reg',
                 RandomForestRegressor(ccp_alpha=0.875, max_depth=50,
                                       max_features='log2',
                                       min_impurity_decrease=0.6,
                                       min_samples_leaf=15, n_estimators=200,
                                       n_jobs=-1, oob_score=True,
                                       random_state=42))])

In [41]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=0.734,RMSE=-50.964
(CV) R2=0.579,RMSE=-62.624


In [42]:
eval.show_result(y_test, model.predict(X_test))

R2=-0.311,RMSE=-47.403


As bad as linear regression. It's the very same model as in FD001.

## Non-linear RUL

In [43]:
model = Pipeline([
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = RandomForestRegressor(random_state=42, n_jobs=-1),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':50})))
    ])

In [44]:
# ~2min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1),
        "trf_reg__regressor__n_estimators": [50,100,200],
        "trf_reg__regressor__criterion": ['squared_error','absolute_error','poisson'],
        "trf_reg__regressor__max_depth": [None,50, 100, 150],
        "trf_reg__regressor__min_samples_split": [2,5,10,15,20],
        "trf_reg__regressor__min_samples_leaf": [1,2,5,10,15,20,50,100],
        "trf_reg__regressor__max_features": ["sqrt", "log2"],
        "trf_reg__regressor__min_impurity_decrease": list(np.arange(0,150)/10),
        "trf_reg__regressor__ccp_alpha": list(np.round(np.linspace(0, 2, 81), decimals=3)),
        "trf_reg__regressor__oob_score": [True, False]
    
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions)
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 24720
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 49
n_resources: 500
Fitting 5 folds for each of 49 candidates, totalling 245 fits
----------
iter: 1
n_candidates: 17
n_resources: 1500
Fitting 5 folds for each of 17 candidates, totalling 85 fits
----------
iter: 2
n_candidates: 6
n_resources: 4500
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 95}, 'trf_reg__regressor__oob_score': True, 'trf_reg__regressor__n_estimators': 50, 'trf_reg__regressor__min_samples_split': 20, 'trf_reg__regressor__min_samples_leaf': 10, 'trf_reg__regressor__min_impurity_decrease': 1.5, 'trf_reg__regressor__max_features': 'log2', 'trf_reg__regressor__max_depth': 150, 'trf_reg__regressor_

Best Model:

```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 95}}

RandomForestRegressor(ccp_alpha=1.075,
                        max_depth=150,
                        max_features='log2',
                        min_impurity_decrease=1.5,
                        min_samples_leaf=10,
                        min_samples_split=20,
                        n_estimators=50,
                        n_jobs=-1,
                        oob_score=True,
                        random_state=42),
```



In [45]:
model = Pipeline([
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = RandomForestRegressor(ccp_alpha=1.075,
                                            max_depth=150,
                                            max_features='log2',
                                            min_impurity_decrease=1.5,
                                            min_samples_leaf=10,
                                            min_samples_split=20,
                                            n_estimators=50,
                                            n_jobs=-1,
                                            oob_score=True,
                                            random_state=42),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':95})))
    ])
model

Pipeline(steps=[('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=RandomForestRegressor(ccp_alpha=1.075,
                                                                            max_depth=150,
                                                                            max_features='log2',
                                                                            min_impurity_decrease=1.5,
                                                                            min_samples_leaf=10,
                                                                            min_samples_split=20,
                                                                            n_estimators=50,
                                                                            n_jobs=-1,
                                                                            oob_score=True,
                                                        

In [46]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=0.863,RMSE=-10.688
(CV) R2=0.842,RMSE=-11.344


In [47]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))

R2=0.812,RMSE=-13.237


Following the results from the decision trees, we have a good result and good generalization.

## PolyFeatures + Linear RUL

In [48]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('tree_reg'  ,   RandomForestRegressor(random_state=42, n_jobs=-1)),
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures()),
                ('tree_reg',
                 RandomForestRegressor(n_jobs=-1, random_state=42))])

In [49]:
# ~6min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False],
        "tree_reg__n_estimators": [50,100,200],
        "tree_reg__criterion": ['squared_error','absolute_error','poisson'],
        "tree_reg__max_depth": [None, 50, 100, 150],
        "tree_reg__min_samples_split": [2,5,10,15,20],
        "tree_reg__min_samples_leaf": [1,2,5,10,15,20,50,100],
        "tree_reg__max_features": ["sqrt", "log2"],
        "tree_reg__min_impurity_decrease": list(np.arange(0,150)/10),
        "tree_reg__ccp_alpha": list(np.round(np.linspace(0, 2, 81), decimals=3)),
        "tree_reg__oob_score": [True, False]
    }
    model = search.run_HR_GS(model, X_train, y_train, 
                            param_distributions, scorer='r2')
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 24720
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 49
n_resources: 500
Fitting 5 folds for each of 49 candidates, totalling 245 fits
----------
iter: 1
n_candidates: 17
n_resources: 1500
Fitting 5 folds for each of 17 candidates, totalling 85 fits
----------
iter: 2
n_candidates: 6
n_resources: 4500
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'tree_reg__oob_score': True, 'tree_reg__n_estimators': 200, 'tree_reg__min_samples_split': 2, 'tree_reg__min_samples_leaf': 15, 'tree_reg__min_impurity_decrease': 0.6, 'tree_reg__max_features': 'log2', 'tree_reg__max_depth': 50, 'tree_reg__criterion': 'squared_error', 'tree_reg__ccp_alpha': 0.875, 'poly_ft__interaction_only': False, 'poly_ft__include_bias': True, 'poly_ft__degr

Best Model:
```
PolynomialFeatures(degree=1)
RandomForestRegressor(ccp_alpha=0.875, max_depth=50,
                        max_features='log2',
                        min_impurity_decrease=0.6,
                        min_samples_leaf=15, n_estimators=200,
                        n_jobs=-1, oob_score=True,
                        random_state=42))])
```


In [50]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=1)),
    ('tree_reg'  ,   RandomForestRegressor(ccp_alpha=0.875, max_depth=50,
                        max_features='log2',
                        min_impurity_decrease=0.6,
                        min_samples_leaf=15, n_estimators=200,
                        n_jobs=-1, oob_score=True,
                        random_state=42))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=1)),
                ('tree_reg',
                 RandomForestRegressor(ccp_alpha=0.875, max_depth=50,
                                       max_features='log2',
                                       min_impurity_decrease=0.6,
                                       min_samples_leaf=15, n_estimators=200,
                                       n_jobs=-1, oob_score=True,
                                       random_state=42))])

In [51]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=0.731,RMSE=-51.276
(CV) R2=0.581,RMSE=-62.471


In [52]:
eval.show_result(y_test, model.predict(X_test))

R2=-0.314,RMSE=-47.460


No clear benefit from polynomial features in linear RUL, as with the other models. Actually, it worsened the results.

## PolyFeatures + Non-Linear RUL

In [53]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = RandomForestRegressor(random_state=42, n_jobs=-1),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':96})))
])

In [54]:
# ~6min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(70,150,1),
        "trf_reg__regressor__n_estimators": [50,100,200],
        "trf_reg__regressor__criterion": ['squared_error','absolute_error','poisson'],
        "trf_reg__regressor__max_depth": [None, 50, 100, 150],
        "trf_reg__regressor__min_samples_split": [2,5,10,15,20],
        "trf_reg__regressor__min_samples_leaf": [1,2,5,10,15,20,50,100],
        "trf_reg__regressor__max_features": ["sqrt", "log2"],
        "trf_reg__regressor__min_impurity_decrease": list(np.arange(0,150)/10),
        "trf_reg__regressor__ccp_alpha": list(np.round(np.linspace(0, 2, 81), decimals=3)),
        "trf_reg__regressor__oob_score": [True, False]
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions)
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 24720
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 49
n_resources: 500
Fitting 5 folds for each of 49 candidates, totalling 245 fits
----------
iter: 1
n_candidates: 17
n_resources: 1500
Fitting 5 folds for each of 17 candidates, totalling 85 fits
----------
iter: 2
n_candidates: 6
n_resources: 4500
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 99}, 'trf_reg__regressor__oob_score': True, 'trf_reg__regressor__n_estimators': 200, 'trf_reg__regressor__min_samples_split': 2, 'trf_reg__regressor__min_samples_leaf': 5, 'trf_reg__regressor__min_impurity_decrease': 5.6, 'trf_reg__regressor__max_features': 'log2', 'trf_reg__regressor__max_depth': 50, 'trf_reg__regressor__c

Best Model:
```
'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 99}
PolynomialFeatures(degree=3, include_bias=False,
                                    interaction_only=True)
RandomForestRegressor(ccp_alpha=0.15,
                    max_depth=50,
                    max_features='log2',
                    min_impurity_decrease=5.6,
                    min_samples_leaf=5,
                    n_estimators=200,
                    n_jobs=-1,
                    oob_score=True,
                    random_state=42),
```


In [55]:
model = Pipeline([
    ('poly_ft', PolynomialFeatures(degree=3, include_bias=False,
                                    interaction_only=True)),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = RandomForestRegressor(ccp_alpha=0.15,
                        max_depth=50,
                        max_features='log2',
                        min_impurity_decrease=5.6,
                        min_samples_leaf=5,
                        n_estimators=200,
                        n_jobs=-1,
                        oob_score=True,
                        random_state=42),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':99})))
])
model

Pipeline(steps=[('poly_ft',
                 PolynomialFeatures(degree=3, include_bias=False,
                                    interaction_only=True)),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=RandomForestRegressor(ccp_alpha=0.15,
                                                                            max_depth=50,
                                                                            max_features='log2',
                                                                            min_impurity_decrease=5.6,
                                                                            min_samples_leaf=5,
                                                                            n_estimators=200,
                                                                            n_jobs=-1,
                                                                            oob_score=True,
 

In [56]:
# ~3min
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=0.848,RMSE=-11.880
(CV) R2=0.830,RMSE=-12.395


In [57]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))

R2=0.776,RMSE=-15.112


Exactly the same model as FD001.

Same results as the decision tree. Considering that the tree is quicker to train, one should choose the tree over the forest. It also allows us to use the tree as feature selector to other models.